In [ ]:
import os
from ansys.acp.core import launch_acp, Client

In [ ]:
SERVER_BIN_PATH = "C:/ANSYSDev/acp/bin/release_with_debug_info/lib/acp_grpcserver.exe"

In [ ]:
server = launch_acp(SERVER_BIN_PATH)

In [ ]:
client = Client(server=server)

In [ ]:
# TODO: infer from file ending
model = client.import_model(path=os.path.abspath("./class40.cdb"), format="ansys:cdb")

In [ ]:
#Create the model (unit system is SI)
corecell_81kg_5mm = model.create_fabric(name="Corecell 81kg", thickness=0.005, material=model.materials["1"])
corecell_103kg_10mm = model.create_fabric(name="Corecell 103kg", thickness=0.01, material=model.materials["2"])
eglass_ud_02mm = model.create_fabric(name="eglass UD", thickness=0.0002, material=model.materials["3"])

ros_deck = model.create_rosette( name='ros_deck', 
                                origin=(-5.9334, -0.0481, 1.693) )
ros_hull = model.create_rosette( name='ros_hull', 
                                origin = (-5.3711, -0.0506, -0.2551) )
ros_bulkhead = model.create_rosette( name='ros_bulkhead', 
                                    origin = (-5.622, 0.0022, 0.0847), 
                                    dir1 = (0.0, 1.0, 0.0), 
                                    dir2 = (0.0, 0.0, 1.0) )
ros_keeltower = model.create_rosette( name='ros_keeltower', 
                                     origin = (-6.0699, -0.0502, 0.623), 
                                     dir1 = (0.0, 0.0, 1.0) )

oss_deck = model.create_oriented_selection_set(name='oss_deck', 
                                               orientation_point = (-5.3806, -0.0016, 1.6449), 
                                               orientation_direction = (0., 0., -1.), 
                                               element_sets=[model.element_sets['DECK']], 
                                               rosettes=[ros_deck])
                                                             
oss_hull = model.create_oriented_selection_set(name='oss_hull', 
                                               orientation_point = (-5.12, 0.1949, -0.2487), 
                                               orientation_direction = (0., 0., 1.), 
                                               element_sets=[model.element_sets['HULL_ALL']], 
                                               rosettes=[ros_hull])                                                     

oss_bulkhead = model.create_oriented_selection_set(name='oss_bulkhead', 
                                                   orientation_point = (-5.622, -0.0465, -0.094), 
                                                   orientation_direction = (1., 0., 0.), 
                                                   element_sets=[model.element_sets['BULKHEAD_ALL']], 
                                                   rosettes=[ros_bulkhead])

esets = [model.element_sets['KEELTOWER_AFT'],
         model.element_sets['KEELTOWER_FRONT'],
         model.element_sets['KEELTOWER_PORT'],
         model.element_sets['KEELTOWER_STB']
        ]
oss_keeltower = model.create_oriented_selection_set(name='oss_keeltower', 
                                                    orientation_point = (-6.1019, 0.0001, 1.162), 
                                                    orientation_direction = (-1., 0., 0.), 
                                                    element_sets=[esets], 
                                                    rosettes=[ros_keeltower])

def add_ply(mg, name, ply_material, angle, oss):
    #todo: pass angle
    return mg.create_modeling_ply(name=name,
                                  ply_material=ply_material,
                                  oriented_selection_sets = oss)

angles = [-90., -60., -45. -30., 0., 0., 30., 45., 60., 90.]

#define plies for the HULL, DECK and BULKHEAD
for mg_name in ['hull', 'deck', 'bulkhead']:
    mg = model.create_modeling_group(name=mg_name)
    oss_list = [model.oriented_selection_sets["oss_"+mg_name]]
    
    for angle in angles:
        add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, oss_list)
        
    add_ply(mg, "corecell_103kg_10mm", corecell_103kg_10mm, oss_list)
    
    for angle in angles:
        add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, oss_list)
    
#add plies to the keeltower
mg = model.create_modeling_group(name=keeltower)
oss_list = [model.oriented_selection_sets["oss_keeltower"]]
for angle in angles:
    add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, oss_list)
        
add_ply(mg, "corecell_81kg_5mm", corecell_103kg_10mm, oss_list)
    
for angle in angles:
    add_ply(mg, "eglass_ud_02mm_"+str(angle), eglass_ud_02mm, oss_list)

In [ ]:
model.update()

In [ ]:
from ansys.api.acp.v0 import model_pb2, model_pb2_grpc

In [ ]:
model_stub = model._get_stub()

In [ ]:
request = model_pb2.SaveAnalysisModelRequest(resource_path=model._resource_path, path=os.path.abspath("./results/class40_analysis_model.cdb"))

In [ ]:
model_stub.SaveAnalysisModel(request)

In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl


In [ ]:
mapdl = launch_mapdl(additional_switches="-smp")

In [ ]:
mapdl.input("class40_analysis_model.cdb")

In [ ]:
mapdl.allsel()
mapdl.slashsolu()
mapdl.solve()

In [ ]:
mapdl.post1()
mapdl.set("last")
mapdl.post_processing.plot_nodal_displacement(component="NORM")

In [ ]:
mapdl.post_processing.plot_nodal_displacement(component="NORM")